In [ ]:
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns

from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import adam_v2

from sklearn.metrics import classification_report,confusion_matrix

import tensorflow as tf

import cv2
import os

import numpy as np
import pandas as pd
import math

In [ ]:
path = '../input/petfinder-pawpularity-score/'
batches = 15
mode = "train"
img_size=300

def assemble_batch(batch_num, batch_length, mode='train'):
    start_index = batch_num * batch_length
    total_length = scores.shape[0]
    data = []
    if (batch_num == batches - 1):
        if (total_length % batches > 0):
            batch_length = batch_length + (total_length % batches)
    end_index = start_index + batch_length
    for x in range(start_index, end_index):
        entry = scores.iloc[x]
        try:
            img_arr = cv2.imread(os.path.join(path + 'train', entry.Id + '.jpg'))[...,::-1] 
            resized_arr = cv2.resize(img_arr, (img_size, img_size)) 
            data.append([resized_arr, entry])
        except Exception as e:
            print(e)
    print(f'Batch {batch_num+1} assembled. Length: {batch_length}')
    return np.array(data, dtype=object)
    

In [ ]:
model = Sequential()

model.add(Conv2D(32,3,padding="same", activation="relu", input_shape=(img_size,img_size,3)))
model.add(MaxPool2D())

model.add(Conv2D(32, 3, padding="same", activation="relu"))
model.add(MaxPool2D())

model.add(Conv2D(64, 3, padding="same", activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(180,activation="relu"))
model.add(Dense(128,activation="softsign"))
model.add(Dense(64,activation="softplus"))
model.add(Dense(13, activation="softmax"))

model.summary()

In [ ]:
opt = tf.keras.optimizers.RMSprop(learning_rate=0.005)
import keras.backend as K
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

model.compile(optimizer = opt , loss = 'squared_hinge' , metrics = ['CategoricalHinge', 'RootMeanSquaredError', 'CategoricalCrossentropy'])

In [ ]:
scores = pd.read_csv(path + mode + '.csv')
batch_length = math.floor(scores.shape[0] / batches)
print(f'Total Length: {scores.shape[0]} | Batches: {batches} | Batch Length: {batch_length}')

In [ ]:
lastLoss = 999
for z in range(batches):
    print(f'Starting Batch {z+1} out of {batches}')
    train = assemble_batch(z, batch_length)
    x_train = []
    y_train = []

    for feature, labels in train:
        x_train.append(feature)
        y_train.append(np.array(labels[1:]))

    del train    
        
    for x in range(len(y_train)):
        for y in range(len(y_train[x])):
            y_train[x][y] = float(y_train[x][y])
            if (y == len(y_train[x] - 1)):
                y_train[x][y] = y_train[x][y] / 100
        y_train[x] = np.array(y_train[x])

    x_train = np.array(x_train) / 255
    x_train.reshape(-1, img_size, img_size, 1)
    y_train = np.asarray(y_train).astype('float32')
    
    t_model = model
    e = 10
    passing = 0
    
    while (passing == 0):
        history = model.fit(x_train,y_train,epochs = e)
        l_hist = history.history['root_mean_squared_error']     
        trend = 0
        for x in range(len(l_hist)):
            if (x > 0):
                if (l_hist[x] < lastLoss):
                    trend = trend - 1
                else:
                    trend = trend + 1
            lastLoss = l_hist[x]
                    
        if (trend >= 0):
            e = 10
            passing = 1
        elif (e > 1):
            if (l_hist[0] < l_hist[len(l_hist) - 1]):
                e = 10
                passing = 1
            else:
                e = e - 1
                model = t_model
        else:
            e = 10
            passing = 1
        
    del x_train
    del y_train

In [ ]:
model2 = Sequential()
model2.add(model)
model2.add(Dense(1, activation="softmax"))
model2.summary()

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))
model2.compile(optimizer = opt , loss = root_mean_squared_error , metrics = ['CategoricalHinge', 'RootMeanSquaredError', 'accuracy'])

In [ ]:
lastLoss = 999
for z in range(batches):
    print(f'Starting Batch {z+1} out of {batches}')
    train = assemble_batch(z, batch_length)
    x_train = []
    y_train = []

    for feature, labels in train:
        x_train.append(feature)
        y_train.append(np.array(labels[-1:]))

    del train    
        
    for x in range(len(y_train)):
        for y in range(len(y_train[x])):
            y_train[x][y] = float(y_train[x][y])
            y_train[x][y] = y_train[x][y] / 100
        y_train[x] = np.array(y_train[x])

    x_train = np.array(x_train) / 255
    x_train.reshape(-1, img_size, img_size, 1)
    y_train = np.asarray(y_train).astype('float32')
    
    t_model = model2
    e = 10
    passing = 0
    
    while (passing == 0):
        history = model2.fit(x_train,y_train,epochs = e)
        l_hist = history.history['loss']     
        trend = 0
        for x in range(len(l_hist)):
            if (x > 0):
                if (l_hist[x] > lastLoss):
                    trend = trend - 1
                else:
                    trend = trend + 1
            lastLoss = l_hist[x]
                    
        if (trend >= 0):
            e = 10
            passing = 1
        elif (e > 1):
            e = e - 1
            model2 = t_model
        else:
            e = 10
            passing = 1
        
    del x_train
    del y_train

In [ ]:
test_data = []
test_results = []

for filename in os.listdir(path + 'test'):
    try:
        img_arr = cv2.imread(os.path.join(path + 'test', filename))[...,::-1] 
        resized_arr = cv2.resize(img_arr, (img_size, img_size)) 
        test_data.append(resized_arr)
        
    except Exception as e:
        print(e)
        
test_data = np.array(test_data) / 255
test_data.reshape(-1, img_size, img_size, 1)
test_results.append(model2.predict(test_data))
print(test_results)